# Relax Challange

In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
#load data
dfuserengagement = pd.read_csv('Datasets/takehome_user_engagement.csv')

In [3]:
dfuserengagement.shape

(207917, 3)

In [4]:
dfuserengagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [5]:
dftakehome = pd.read_csv('Datasets/takehome_users.csv', encoding="latin-1")

In [6]:
dftakehome.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [7]:
dftakehome.shape

(12000, 10)

In [8]:
#convert time stamp to datetime
dfuserengagement.time_stamp = pd.to_datetime(dfuserengagement.time_stamp)
dfuserengagement.index=dfuserengagement.time_stamp
dfuserengagement.drop(labels='time_stamp',axis=1,inplace=True)

In [9]:
dfuserengagement.head()

,user_id,visited
time_stamp,,
2014-04-22 03:53:30,1,1
2013-11-15 03:45:04,2,1
2013-11-29 03:45:04,2,1
2013-12-09 03:45:04,2,1
2013-12-25 03:45:04,2,1


In [10]:
#Who are actually used the application and the number of signups 

print('There are {} unique users who used the application.'\
          .format(len(list(dfuserengagement['user_id'].unique()))))
print('There are {} unique signed up'\
          .format(len(list(dftakehome['object_id'].unique()))))

There are 8823 unique users who used the application.
There are 12000 unique signed up


In [11]:
#resample to 1 week period and aggregate
dfsum = dfuserengagement.groupby([pd.Grouper(freq='W'),'user_id']).sum()

In [12]:
#find out adopted users
dfadopt = dfsum[dfsum.visited>=3].unstack(level=1).melt()
dfadoptedusers = pd.DataFrame(dfadopt.user_id.unique(),
                             index=range(dfadopt.user_id.unique().shape[0]),columns=['user_id'])

In [13]:
#merge adopted users dataframe to takehome users
dfmerge = dftakehome.merge(dfadoptedusers,how='inner',left_on='object_id',right_on='user_id')
dfmerge.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id
0,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2
1,10,2013-01-16 22:08:03,Santos Carla,CarlaFerreiraSantos@gustr.com,ORG_INVITE,1.401833e+09,1,1,318,4143.0,10
2,20,2014-03-06 11:46:38,Helms Mikayla,lqyvjilf@uhzdq.com,SIGNUP,1.401364e+09,0,0,58,NaN,20
3,33,2014-03-11 06:29:09,Araujo José,JoseMartinsAraujo@cuvox.de,GUEST_INVITE,1.401518e+09,0,0,401,79.0,33
4,42,2012-11-11 19:05:07,Pinto Giovanna,GiovannaCunhaPinto@cuvox.de,SIGNUP,1.401045e+09,1,0,235,NaN,42


In [14]:
drop_cols = list(dfmerge.columns[0:4])
drop_cols.append('user_id')
dfmerge = dfmerge.drop(drop_cols,axis=1)

In [15]:
dfmerge.head()

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.0
1,ORG_INVITE,1.401833e+09,1,1,318,4143.0
2,SIGNUP,1.401364e+09,0,0,58,NaN
3,GUEST_INVITE,1.401518e+09,0,0,401,79.0
4,SIGNUP,1.401045e+09,1,0,235,NaN


In [16]:
dfmerge['invited_by_user_id'].mean()

5966.110159118727

In [17]:
#fill nan values in invited by user id with mean
dfmerge['invited_by_user_id'].fillna((dfmerge['invited_by_user_id'].mean()), inplace=True)

In [18]:
dfmerge.head()

,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,ORG_INVITE,1.396238e+09,0,0,1,316.000000
1,ORG_INVITE,1.401833e+09,1,1,318,4143.000000
2,SIGNUP,1.401364e+09,0,0,58,5966.110159
3,GUEST_INVITE,1.401518e+09,0,0,401,79.000000
4,SIGNUP,1.401045e+09,1,0,235,5966.110159


In [19]:
dfmergedum= pd.get_dummies(dfmerge, columns = ['creation_source'])

In [20]:
dfmergedum.head()

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
0,1.396238e+09,0,0,1,316.000000,0,1,0,0,0
1,1.401833e+09,1,1,318,4143.000000,0,1,0,0,0
2,1.401364e+09,0,0,58,5966.110159,0,0,0,1,0
3,1.401518e+09,0,0,401,79.000000,1,0,0,0,0
4,1.401045e+09,1,0,235,5966.110159,0,0,0,1,0


In [21]:
dfmergedum.columns

Index(['last_session_creation_time', 'opted_in_to_mailing_list',
       'enabled_for_marketing_drip', 'org_id', 'invited_by_user_id',
       'creation_source_GUEST_INVITE', 'creation_source_ORG_INVITE',
       'creation_source_PERSONAL_PROJECTS', 'creation_source_SIGNUP',
       'creation_source_SIGNUP_GOOGLE_AUTH'],
      dtype='object')

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [23]:
#scale the data
scaler = StandardScaler()
features = scaler.fit_transform(dfmergedum)


In [24]:
#check the null values
dfmergedum.isnull().sum()

last_session_creation_time            0
opted_in_to_mailing_list              0
enabled_for_marketing_drip            0
org_id                                0
invited_by_user_id                    0
creation_source_GUEST_INVITE          0
creation_source_ORG_INVITE            0
creation_source_PERSONAL_PROJECTS     0
creation_source_SIGNUP                0
creation_source_SIGNUP_GOOGLE_AUTH    0
dtype: int64

In [25]:
#fit Principle Components
pca = PCA()
components = pca.fit_transform(features)

In [26]:
np.sum(pca.explained_variance_ratio_[0:8])

0.9492353934807105

__INTERPRETATION:__ The first eight principal components acount for about 95% of the variation in the data. I also want to ched check the relative importance of all features. 

In [27]:
dfpca = pd.DataFrame(pca.components_,columns=dfmergedum.columns,index=['PC-1','PC-2','PC-3','PC-4','PC-5','PC-6','PC-7','PC-8','PC-9','PC-10'])
#correlation with principal components
best_features = np.absolute(dfpca[np.absolute(dfpca) > 0.1])
best_features.head(10)

,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH
PC-1,NaN,0.688682,0.688362,NaN,NaN,NaN,0.149579,NaN,NaN,NaN
PC-2,NaN,0.127596,0.125664,NaN,NaN,0.430849,0.805233,0.107420,0.300694,0.145282
PC-3,0.222021,NaN,NaN,0.140770,NaN,0.676196,NaN,NaN,0.662868,0.114232
PC-4,NaN,NaN,NaN,0.253930,NaN,0.238227,0.128119,NaN,0.384448,0.834612
PC-5,0.175841,NaN,NaN,0.222877,NaN,0.178591,NaN,0.882637,0.260703,0.168497
PC-6,0.358998,NaN,NaN,0.546939,0.668172,NaN,NaN,0.229352,NaN,0.239053
PC-7,0.778485,NaN,NaN,0.121613,0.580325,NaN,NaN,0.110022,0.120813,NaN
PC-8,0.421613,NaN,NaN,0.736938,0.443165,0.134587,NaN,NaN,0.191042,0.109882
PC-9,NaN,0.704926,0.705633,NaN,NaN,NaN,NaN,NaN,NaN,NaN
PC-10,NaN,NaN,NaN,NaN,NaN,0.477341,0.540903,0.345224,0.448295,0.399277


In [28]:
best_features.head(10).sum(axis=0).sort_values(ascending=False)

creation_source_SIGNUP                2.368863
creation_source_GUEST_INVITE          2.135791
org_id                                2.023067
creation_source_SIGNUP_GOOGLE_AUTH    2.010836
last_session_creation_time            1.956958
invited_by_user_id                    1.691662
creation_source_PERSONAL_PROJECTS     1.674655
creation_source_ORG_INVITE            1.623835
opted_in_to_mailing_list              1.521204
enabled_for_marketing_drip            1.519659
dtype: float64

__INTERPRETATION:__ As seen above, creation_source_SIGNUP, creation_source_GUEST_INVITE, org_id, creation_source_SIGNUP_GOOGLE_AUTH, and last_session_creation_time components account for most of the data variation in the dataset of adopted users, meaning that these variables are highly predictive of future user adoption.  